In [1]:
import pandas as pd

In [2]:
df_train = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')

<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject


In [3]:
df_hw = df_train.copy()
print(f'Q1. The number of columns is:  {len(df_hw.columns)}')

Q1. The number of columns is:  19


In [4]:
df_hw['duration'] = df_hw.tpep_dropoff_datetime - df_hw.tpep_pickup_datetime
df_hw.duration = df_hw.duration.apply(lambda td: td.total_seconds() / 60)
print(f'Q2. The standard deviation of the trips in January is: {df_hw.duration.std()}')

Q2. The standard deviation of the trips in January is: 42.594351241920904


In [5]:
filtered = df_hw[(df_hw.duration >= 1) & (df_hw.duration <= 60)]
print(f'Q3. The fraction of the number of trips left is:  {len(filtered)/len(df_hw)}.')

Q3. The fraction of the number of trips left is:  0.9812202822125979.


In [18]:
categorical = ['PULocationID', 'DOLocationID']
df_hw[categorical] = df_hw[categorical].astype(str)
training_dicts = df_hw[categorical].to_dict(orient='records')
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
X_training = dv.fit_transform(training_dicts)
print(f'Q4. The number of columns after one-hot encoding is: {X_training.shape[1]}.')

Q4. The number of columns after one-hot encoding is: 518.


In [7]:
df_train.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
def preprocess(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[df.duration >= 1]
    df = df[df.duration <= 60]
    return df

In [9]:
df_train = preprocess(df_train)

In [10]:
df_val = preprocess(df_val)

In [11]:
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

In [12]:
train_dicts = df_train[categorical].to_dict(orient='records')

In [13]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print(f'Q5. This is the root mean squared error for the training set: {rmse}')
val_pred = lr.predict(X_val)

val_rmse = mean_squared_error(y_val, val_pred, squared=False)
print(f'Q6. This is the root mean squared error for the validation set: {val_rmse}')

Q5. This is the root mean squared error for the training set: 7.64926180044339
Q6. This is the root mean squared error for the validation set: 7.8118236307935


In [17]:
! python --version

Python 3.9.18
